# Shopee - Detecting Product Similarity (Texts, Images, and Phash)
The task of this competition is to "build a model that predicts which items are the same products." We tackle this task by implementing KNN to group similar products by their title and images, and [image phash](https://en.wikipedia.org/wiki/Perceptual_hashing). We also use RAPIDS to accelerate KNN computations on the GPU. A huge thanks to the authors of the following kernel as well.
* Chris Deoette - [[PART 2] - RAPIDS TfidfVectorizer - [CV 0.700]](https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700/notebook#Use-Image-Embeddings)

# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import tensorflow as tf
import sys
import gc
import math
from tqdm import tqdm

## RAPIDS Installation & Imports
Kaggle has the 0.16 version of RAPIDS installed, but this version's `fit` and `fit_transform` methods [do not support sparse cupy matrices](https://www.kaggle.com/c/shopee-product-matching/discussion/230152) like more recent versions. It only supports dense matrices which require much more memory, so we install version 0.18 in order to save GPU memory.

In [ ]:
!cp ../input/rapids/rapids.0.18.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
print('RAPIDS 0.18 installation complete')

In [ ]:
import cuml, cudf, cupy
from cuml.feature_extraction.text import TfidfVectorizer

# Examine Data
## Load Data

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
test = pd.read_csv('../input/shopee-product-matching/test.csv')
submission = pd.read_csv('../input/shopee-product-matching/sample_submission.csv')

print('Train shape: {}'.format(train.shape))
train.head()

In [ ]:
print('Test shape: {}'.format(test.shape))
test.head()

In [ ]:
print('Submission shape: {}'.format(submission.shape))
submission.head()

## Check Samples

In [ ]:
def show_samples(data_df, num, title, img_path = '../input/shopee-product-matching/train_images/'):
    cols = 6
    rows = math.ceil(num / cols)
    height = 5 * rows
    fig, axs = plt.subplots(rows, cols, figsize = (20, height))
    fig.suptitle('{}\n'.format(title), fontsize = 25)
    for row in range(rows):
        for col in range(cols):     
            if rows > 1:
                ax = axs[row, col]
            else:
                ax = axs[col]
                
            ax.axis('off')
            
            df_row = col + row * cols # examine row by row for dataframe
            title = data_df.iloc[df_row, 3]
            title_with_return = ''
            for i, ch in enumerate(title):
                title_with_return += ch
                if (i != 0) & (i % 20 == 0): 
                    title_with_return += '\n'
            ax.set_title(title_with_return)
            
            posting_id = data_df.iloc[df_row, 1]
            img_bgr = cv2.imread(img_path + posting_id)
            img_rgb = cv2.cvtColor(img_bgr,  cv2.COLOR_BGR2RGB) # convert image back to RGB for visualization
            ax.imshow(img_rgb)    

In [ ]:
show_samples(data_df = train, num = 12, title = 'Product Samples')

## Check Duplicates

In [ ]:
groups = train['label_group'].value_counts()

plt.figure(figsize = (20, 5))
plt.plot(np.arange(len(groups)),groups.values)
plt.title('Number of Duplicates',size = 25)
plt.ylabel('Count', size = 15)
plt.xlabel('Duplicate Products', size = 15)
plt.show()

plt.figure(figsize = (20, 5))
plt.bar(groups.index.values[:50].astype('str'), groups.values[:50])
plt.xticks(rotation = 45)
plt.title('Top 50 Duplicates', size = 25)
plt.ylabel('Count', size = 15)
plt.xlabel('Label Group', size = 15)
plt.show()

### Top Duplicates

In [ ]:
TOP_N = 3
COUNT = 6

top_duplicates = train['label_group'].value_counts().index[:TOP_N]
for i, duplicate_label in enumerate(top_duplicates):
    duplicate_label_df = train.loc[train['label_group'] == duplicate_label]
    duplicate_title = 'Top #{} Product With Duplicates'.format(i + 1)
    show_samples(data_df = duplicate_label_df, num = COUNT, title = duplicate_title)

# Setup RAPIDS
We setup RAPIDS by distributing the memory limits for TensorFlow and RAPIDS. Since we perform the KNN computations on the GPU, we distribute more memory to RAPIDS.
## Setup GPU Memory

In [ ]:
LIMIT = 1 # 1 GB of GPU for TensorFlow, 15 GB of GPU for RAPIDS (GPU total: 16 GB)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 1024 * LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print('GPU is online.')
    except RuntimeError as e:
        print(e)
else:
    print('GPU is offline.')
print('TensorFlow GPU memory limit set to {} GB'.format(LIMIT))
print('RAPIDS GPU memory limit set to {} GB'.format(16 - LIMIT))

***
# Title Predictions
## Extract Text Embeddings with Tf-idf Vectorizer
We use tf-idf [to scale down the impact of tokens that occur very frequently in a given corpus](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer). This helps the model to disregard less informative tokens and emphasize more informative ones.

In [ ]:
def get_text_embeddings(text_df):
    text_gf = cudf.from_pandas(text_df)
    text_embed_model = TfidfVectorizer(
        stop_words = 'english', 
        binary = True) # binary occurence since short titles
    text_embeddings = text_embed_model.fit_transform(text_gf)
    
    return text_embeddings

In [ ]:
title_embeddings = get_text_embeddings(train['title'])
title_embeddings.shape

## Get Title Distances with KNN
We get the distances and indices of the 50 closest neighbors to each text embedding of a product's title. ([Group sizes were capped at 50, so there is no benefit to predict more than 50 matches](https://www.kaggle.com/c/shopee-product-matching/overview/evaluation)) Also, for text classification the cosine distance metric performs well since it disregards the magnitudes of the embeddings, meaning it is better at capturing similarities across varying lengths of text which is our case.

In [ ]:
def knn_predict_embeddings(embeddings, metric, n_neighbors = 50):
    BATCH_SIZE = 1024 * 4
    n_texts = embeddings.shape[0]
    n_batches = math.ceil(n_texts / BATCH_SIZE)
    
    knn_model = cuml.NearestNeighbors(n_neighbors = n_neighbors, metric = metric)
    knn_model.fit(embeddings)
    
    embed_distances = np.zeros((n_texts, n_neighbors))
    embed_indices = np.zeros((n_texts, n_neighbors))

    with cuml.using_output_type('numpy'): # to output as numpy arrays
        for i in tqdm(range(n_batches)):
            a = i * BATCH_SIZE
            b = min((i + 1) * BATCH_SIZE, n_texts)
            distances, indices = knn_model.kneighbors(embeddings[a:b])
            embed_distances[a:b] = distances
            embed_indices[a:b] = indices
        
    return embed_distances, embed_indices

In [ ]:
title_distances, title_indices = knn_predict_embeddings(title_embeddings, 'cosine')
del title_embeddings

### Title Distance Samples
We check the distances of each neighbor from a number of samples.

In [ ]:
def show_distance_samples(dist_type, distances, indices,  metric = '', n_items = 3, n_neighbors = 5, random = False):
    for i in range(n_items): 
        if random: 
            i = np.random.randint(0, len(train))  
        item_title = train.loc[indices[i, 0], 'title']
        
        plt.figure(figsize=(15,3))
        plt.plot(np.arange(50), distances[i,],'o-')
        plt.title('{} Distances From "{}" to Neighbors'.format(dist_type, item_title),size=16)
        plt.ylabel('{} Distance'.format(metric),size = 20)
        plt.xlabel('Closest 50 Neighbors',size = 15)
        plt.show()

        print(train.loc[indices[i, :n_neighbors],['title','label_group']] )

In [ ]:
show_distance_samples('Title', title_distances, title_indices, 'Cosine')

### Extract Match Title Distances
We extract the distances of neighbors which are true duplicate matches of each product by grouping them according to each `label_group`.

In [ ]:
def show_match_distances(title, distances, indices):
    match_distances = []
    for i in tqdm(range(distances.shape[0])):
        for j in range(distances.shape[1]):
            if train.loc[i, 'label_group'] == train.loc[indices[i, j], 'label_group']:
                match_distances.append(distances[i, j])
    if type(match_distances) != list:
        match_distances = [dist for dist_list in match_distances for dist in dist_list]
    plt.hist(match_distances, bins = 20)
    plt.title('{} Neighbor Distance Distribution'.format(title), size = 20)
    plt.show()
    print(pd.DataFrame(match_distances, columns=['distances']).describe())
    del match_distances

def get_match_distances(distances, indices):
    match_distances = []
    for i in tqdm(range(distances.shape[0])):
        for j in range(distances.shape[1]):
            if train.loc[i, 'label_group'] == train.loc[indices[i, j], 'label_group']:
                match_distances.append(distances[i, j])
    return match_distances

def show_match_distance_distribution(title, match_distances):
    if type(match_distances) != list:
        match_distances = [dist for dist_list in match_distances for dist in dist_list]
    plt.hist(match_distances, bins = 20)
    plt.title('{} Neighbor Distance Distribution'.format(title), size = 20)
    plt.show()
    print(pd.DataFrame(match_distances, columns=['distances']).describe())

In [ ]:
show_match_distances('Title', title_distances, title_indices)

## Determine Best Title Distance
We need to decide upon a maximum distance in order to predict whether a certain neighbor is a duplicate of the product or not. To do so we calculate the mean **f1 score,** which is this competition's evluation metric, of the title predictions based on different maximum distances.

### Setup Target Values
We setup the target values by grouping every product to each `label_group`.

In [ ]:
target_dict = train.groupby('label_group')['posting_id'].agg('unique').to_dict()
train['target'] = train['label_group'].map(target_dict)

### F1 Score Metric
We create a function to calculate the f1 score of each prediction.

In [ ]:
def get_f1_score(row, pred_col):
    true_preds = len(np.intersect1d(row['target'], row[pred_col + '_pred']))
    f1_score = (2 * true_preds) / (len(row['target']) + len(row[pred_col + '_pred']))
           # = 2 * TP / ((2 * TP) + FN + FP) 
           # = 2 * TP / (TP + TP + FN + FP) 
           # = 2 * TP / ((TP + FN) + (TP + FP)) 
           # = 2 * TP / (len(target) + len(predictions))
    return f1_score

### Make Predictions and Choose Max Title Distance
We make predictions on various maximum distances and find the distance obtaining the highest f1 score.

In [ ]:
def get_predictions(data_df, distances, indices, max_distance):
    predictions = []
    for i in tqdm(range(distances.shape[0])): 
        pred_distances = np.where(distances[i] < max_distance)[0]
        pred_indices = indices[i, pred_distances]
        row_pred = data_df.iloc[pred_indices]['posting_id'].values.tolist()
        predictions.append(row_pred)
    return predictions

def show_max_distance_f1_scores(distances, indices, max_distances, pred_col):
    for max_distance in max_distances:
        train[pred_col + '_pred'] = get_predictions(train, distances, indices, max_distance)
        train['f1_score_' + pred_col] = train.apply(get_f1_score, axis = 'columns', pred_col = pred_col)
        print('{} max distance: {}, {} f1 score = {}'.format(pred_col, max_distance, pred_col, train['f1_score_' + pred_col].mean()))
    train.drop(columns = 'f1_score_' + pred_col, inplace = True)

In [ ]:
MAX_TITLE_DISTANCES = [0.46861,
                       0.46862,
                       0.46863,
                       0.46864,
                       0.46865]

show_max_distance_f1_scores(title_distances, title_indices, MAX_TITLE_DISTANCES, 'title')

The maximum distance value of `0.46863` obtained the highest f1 score, so we use this distance for our final predictions.

In [ ]:
MAX_TITLE_DISTANCE = 0.46863 # best f1 score of 0.6610465706375579

## Attach Title Predictions

In [ ]:
train['title_pred'] = get_predictions(train, title_distances, title_indices, MAX_TITLE_DISTANCE)
del title_distances, title_indices
_ = gc.collect()
train.head()

***
# Image Predictions
## Create Data Generator
We create a data generator to pass on the image data to our embedding model in batches to prevent memory errors.

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size, batch_size, path): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange(len(self.df))
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(((len(self.df)) % self.batch_size) != 0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        X = self.__data_generation(indexes)
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes), self.img_size, self.img_size, 3), dtype = 'float32')
        df = self.df.iloc[indexes]
        for i,(index, row) in enumerate(df.iteritems()):
            img = cv2.imread(self.path + row)
            X[i,] = cv2.resize(img, (self.img_size, self.img_size)) 
        return X

## Extract Image Embeddings with Pretrained Model
We use the EfficientNetB0 model to extract image features as vectors to use for KNN. Since submissions must disable internet connection, we need to load the model weights as an input file.

In [ ]:
def get_image_embeddings(image_df, image_path):
    image_embed_model = tf.keras.applications.EfficientNetB0( # image distances most evenly distributed among pretrained models
        weights = '../input/shopee-data/efficientnetb0_notop.h5', # same as calling 'imagenet', but need file import because internet must be disabled
        include_top = False, 
        pooling = 'avg', 
        input_shape = None)
    
    n_images = image_df.shape[0]
    n_features = image_embed_model.layers[-1].output_shape[1] # size of final output layer
    image_embeddings = np.zeros((n_images, n_features))
    
    input_img_size = 256 # input size of EfficientNetB0
    BATCH_SIZE = 1024 * 4
    EPOCHS = math.ceil(n_images / BATCH_SIZE)
    for i in tqdm(range(EPOCHS)):
        a = i * BATCH_SIZE
        b = min((i + 1) * BATCH_SIZE, n_images)

        image_gen = DataGenerator(df = image_df.iloc[a:b], img_size = input_img_size, batch_size = 32, path = image_path)
        batch_embeddings = image_embed_model.predict(image_gen, verbose = 1, use_multiprocessing = True, workers = 4)
        image_embeddings[a:b] = batch_embeddings
    
    return image_embeddings

In [ ]:
TRAIN_PATH = '../input/shopee-product-matching/train_images/'

image_embeddings = get_image_embeddings(train['image'], TRAIN_PATH)
image_embeddings.shape

## Get Image Distances with KNN
Like what we did for the titles, we get the distances and indices of the 50 closest neighbors to each image embedding of a product's image. This time we use euclidean distance.

In [ ]:
image_distances, image_indices = knn_predict_embeddings(image_embeddings, 'euclidean')
del image_embeddings

### Text Distance Samples
We check the distances of each neighbor from a number of samples.

In [ ]:
show_distance_samples('Image', image_distances, image_indices, 'Euclidean')

### Extract Match Image Distances
We extract the distances of neighbors which are true duplicate matches of each product by grouping them according to each `label_group`.

In [ ]:
show_match_distances('Image', image_distances, image_indices)

## Determine Best Image Distance
We need to decide upon a maximum distance in order to predict whether a certain neighbor is a duplicate of the product or not. To do so we calculate the mean **f1 score,** which is this competition's evluation metric, of the image predictions based on different maximum distances.

In [ ]:
MAX_IMAGE_DISTANCES = [6.88564,
                       6.88566,
                       6.88568,
                       6.88570,
                       6.88572]

show_max_distance_f1_scores(image_distances, image_indices, MAX_IMAGE_DISTANCES, 'image')

The maximum distance value of `6.88568` obtained the highest f1 score, so we use this distance for our final predictions.

In [ ]:
MAX_IMAGE_DISTANCE = 6.88568 # best f1 score of 0.6469574990306197

## Attach Image Predictions

In [ ]:
train['image_pred'] = get_predictions(train, image_distances, image_indices, MAX_IMAGE_DISTANCE)
del image_distances, image_indices
_ = gc.collect()
train.head()

***
# Phash Predictions
We use the [perceptual hashes](https://en.wikipedia.org/wiki/Perceptual_hashing) of the product images as duplicate predictions by grouping them together.

In [ ]:
phash_dict = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['phash_pred'] = train.image_phash.map(phash_dict)
del phash_dict
_ = gc.collect()
train.head()

***
# Compute Validation F1 Score
We combine the three predictions we have made to obtain a single match prediction to evalute our model.

In [ ]:
def combine_preds_validation(row):
    pred = np.concatenate([row['title_pred'], row['image_pred'], row['phash_pred']])
    return np.unique(pred)

def combine_preds_submission(row):
    pred = np.concatenate([row['title_pred'], row['image_pred'], row['phash_pred']])
    return ' '.join(np.unique(pred)) # submission must be in 'pred_A pred_B pred_C ...' format

In [ ]:
train['match_pred'] = train.apply(combine_preds_validation, axis = 'columns')
train['f1_score'] = train.apply(get_f1_score, axis = 'columns', pred_col = 'match')
train.head()

In [ ]:
print('validation f1 score = {}'.format(train['f1_score'].mean()))

***
# Get Test Predictions and Submit
We now perform the same process on the test set and submit these predictions to the competition. We are only provided 3 rows of data for the test set and will be provided the rest hidden upon submission. Thus, we adjust the number of neighbors accordingly.

In [ ]:
test_neighbors = min(50, test.shape[0])

title_embeddings = get_text_embeddings(test['title'])

title_distances, title_indices = knn_predict_embeddings(title_embeddings, 'cosine', test_neighbors)
del title_embeddings

test['title_pred'] = get_predictions(test, title_distances, title_indices, MAX_TITLE_DISTANCE)
del title_distances, title_indices
_ = gc.collect()

TEST_PATH = '../input/shopee-product-matching/test_images/'

image_embeddings = get_image_embeddings(test['image'], TEST_PATH)

image_distances, image_indices = knn_predict_embeddings(image_embeddings, 'euclidean', test_neighbors)
del image_embeddings

test['image_pred'] = get_predictions(test, image_distances, image_indices, MAX_IMAGE_DISTANCE)
del image_distances, image_indices
_ = gc.collect()

phash_dict = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['phash_pred'] = test.image_phash.map(phash_dict)
del phash_dict
_ = gc.collect()

test['matches'] = test.apply(combine_preds_submission, axis = 'columns')
test.head()

In [ ]:
test[['posting_id','matches']].to_csv('submission.csv',index = False)